In [1]:
!pip install geopy

In [2]:
import pandas as pd
from geopy.geocoders import Nominatim

In [3]:
# setup geopy
geolocator = Nominatim(user_agent="preprocessing_affordable_units")

In [4]:
def find_coordinates(df_row):
    location = geolocator.geocode(df_row['Building Address'])
    try:
        return (location.latitude, location.longitude)
    except:
        return (pd.NA, pd.NA)

In [13]:
# add coordinates to main dataset using geopy
df = pd.read_csv('../data/affordable_units.csv')[:50]
df['coordinates'] = df.apply(find_coordinates, axis=1)
df.to_csv('../data/preprocessed_affordable_units.csv')